In [71]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import os
import re
import warnings

warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [72]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)

In [73]:
path1 = 'A榜数据/主表数据/'
path2 = 'A榜数据/其他数据表/'

In [74]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem)) 
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

### A产品表1

In [75]:
g_data = pd.read_csv(path2 + 'g.csv')
g_data.head(2)

,prod_code,g1,g2,g3,g4,g5,g6,g7,g8,g9
0,ZXGRHLA2021059,0,1,1,1,0,0,0,364,209912
1,GRSHB201902005,0,1,1,1,1,0,0,1,209912


In [76]:
g_columns = ['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否滚存', '是否允许变更分红方式', '产品品种', '持有天数', '数据日期']

In [77]:
g_data.columns = g_columns
g_data.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否滚存,是否允许变更分红方式,产品品种,持有天数,数据日期
0,ZXGRHLA2021059,0,1,1,1,0,0,0,364,209912
1,GRSHB201902005,0,1,1,1,1,0,0,1,209912


In [78]:
g_data['数据日期'] = g_data['数据日期'].astype('object')

In [79]:
g_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6557 entries, 0 to 6556
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prod_code   6557 non-null   object
 1   计价类型        6557 non-null   int64 
 2   周期类型        6557 non-null   int64 
 3   模式          6557 non-null   int64 
 4   风险等级        6557 non-null   int64 
 5   是否滚存        6557 non-null   int64 
 6   是否允许变更分红方式  6557 non-null   int64 
 7   产品品种        6557 non-null   int64 
 8   持有天数        6557 non-null   int64 
 9   数据日期        6557 non-null   object
dtypes: int64(8), object(2)
memory usage: 512.4+ KB


In [80]:
g_data.drop(['是否滚存', '产品品种'], axis=1, inplace=True)

### A产品表2

In [81]:
k_data = pd.read_csv(path2 + 'k.csv')
k_data.head(2)

,prod_code,k1,k2,k3,k4,k5,k6,k7,k8,k9,k10,k11
0,GRSHA2020423,NaN,NaN,NaN,2,3.0,1,0.032600,0.0326,0.0011,5.0,209912
1,GRHLA2021142,NaN,NaN,NaN,2,NaN,1,0.041525,0.0431,0.0158,NaN,209912


In [82]:
k_columns = ['prod_code', '募集方式', '管理方式', '业务模式', '收益特点', '期限', '投资模式', 
             '预期收益率', '最高收益率', '最低收益率', '展示等级', '数据日期']

In [83]:
k_data.columns = k_columns
k_data.head(2)

,prod_code,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,最高收益率,最低收益率,展示等级,数据日期
0,GRSHA2020423,NaN,NaN,NaN,2,3.0,1,0.032600,0.0326,0.0011,5.0,209912
1,GRHLA2021142,NaN,NaN,NaN,2,NaN,1,0.041525,0.0431,0.0158,NaN,209912


In [84]:
k_data['数据日期'] = k_data['数据日期'].astype('object')

In [85]:
k_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22874 entries, 0 to 22873
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prod_code  22874 non-null  object 
 1   募集方式       314 non-null    float64
 2   管理方式       314 non-null    float64
 3   业务模式       314 non-null    float64
 4   收益特点       22874 non-null  int64  
 5   期限         2585 non-null   float64
 6   投资模式       22874 non-null  int64  
 7   预期收益率      22874 non-null  float64
 8   最高收益率      22874 non-null  float64
 9   最低收益率      22874 non-null  float64
 10  展示等级       19859 non-null  float64
 11  数据日期       22874 non-null  object 
dtypes: float64(8), int64(2), object(2)
memory usage: 2.1+ MB


In [86]:
k_data.drop(['最高收益率', '最低收益率'], axis=1, inplace=True)

### A产品总表

In [87]:
a_prod = pd.merge(g_data, k_data, on=['prod_code', '数据日期'], how='outer')
a_prod['prod_class'] = 1
a_prod.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class
0,ZXGRHLA2021059,0.0,1.0,1.0,1.0,0.0,364.0,209912.0,NaN,NaN,NaN,2.0,NaN,1.0,0.04268,NaN,1
1,GRSHB201902005,0.0,1.0,1.0,1.0,0.0,1.0,209912.0,NaN,NaN,NaN,2.0,NaN,1.0,0.02735,5.0,1


In [88]:
a_prod['计价类型'].fillna(0, inplace=True)
a_prod['周期类型'].fillna(1, inplace=True)
a_prod['模式'].fillna(1, inplace=True)
a_prod['风险等级'].fillna(1, inplace=True)
a_prod['是否允许变更分红方式'].fillna(0, inplace=True)
a_prod['投资模式'].fillna(1, inplace=True)
a_prod['募集方式'].fillna(0, inplace=True)
a_prod['管理方式'].fillna(2, inplace=True)
a_prod['业务模式'].fillna(1, inplace=True)
a_prod['展示等级'].fillna(5, inplace=True)

In [89]:
a_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22904 entries, 0 to 22903
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   22904 non-null  object 
 1   计价类型        22904 non-null  float64
 2   周期类型        22904 non-null  float64
 3   模式          22904 non-null  float64
 4   风险等级        22904 non-null  float64
 5   是否允许变更分红方式  22904 non-null  float64
 6   持有天数        6557 non-null   float64
 7   数据日期        22904 non-null  float64
 8   募集方式        22904 non-null  float64
 9   管理方式        22904 non-null  float64
 10  业务模式        22904 non-null  float64
 11  收益特点        22874 non-null  float64
 12  期限          2585 non-null   float64
 13  投资模式        22904 non-null  float64
 14  预期收益率       22874 non-null  float64
 15  展示等级        22904 non-null  float64
 16  prod_class  22904 non-null  int64  
dtypes: float64(15), int64(1), object(1)
memory usage: 3.1+ MB


In [90]:
a_prod.nunique()

prod_code     22899
计价类型              1
周期类型              1
模式                1
风险等级              2
是否允许变更分红方式        1
持有天数            319
数据日期             30
募集方式              1
管理方式              1
业务模式              1
收益特点              1
期限                6
投资模式              1
预期收益率           213
展示等级              5
prod_class        1
dtype: int64

In [91]:
a_prod.describe()

,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class
count,22904.0,22904.0,22904.0,22904.000000,22904.0,6557.000000,22904.000000,22904.0,22904.0,22904.0,22874.0,2585.000000,22904.0,22874.000000,22904.000000,22904.0
mean,0.0,1.0,1.0,1.000699,0.0,140.606222,209792.793791,0.0,2.0,1.0,2.0,4.682012,1.0,0.038843,4.965596,1.0
std,0.0,0.0,0.0,0.037373,0.0,120.693229,969.745922,0.0,0.0,0.0,0.0,1.097530,0.0,0.005822,0.347953,0.0
min,0.0,1.0,1.0,1.000000,0.0,1.000000,201808.000000,0.0,2.0,1.0,2.0,2.000000,1.0,0.013700,1.000000,1.0
25%,0.0,1.0,1.0,1.000000,0.0,37.000000,209912.000000,0.0,2.0,1.0,2.0,4.000000,1.0,0.035225,5.000000,1.0
50%,0.0,1.0,1.0,1.000000,0.0,99.000000,209912.000000,0.0,2.0,1.0,2.0,5.000000,1.0,0.038900,5.000000,1.0
75%,0.0,1.0,1.0,1.000000,0.0,200.000000,209912.000000,0.0,2.0,1.0,2.0,6.000000,1.0,0.042575,5.000000,1.0
max,0.0,1.0,1.0,3.000000,0.0,371.000000,209912.000000,0.0,2.0,1.0,2.0,7.000000,1.0,0.077540,5.000000,1.0


### B产品表1

In [92]:
h_data = pd.read_csv(path2 + 'h.csv')
h_data.head(2)

,prod_code,h1,h2,h3,h4,h5,h6,h7,h8
0,ZYGR2016286,0,1,1,2,0,0,NaN,209912
1,ZYGR2015103,0,1,1,2,1,0,NaN,209912


In [93]:
h_columns = ['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '产品品种', '模式2', '数据日期']

In [94]:
h_data.columns = h_columns
h_data.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,产品品种,模式2,数据日期
0,ZYGR2016286,0,1,1,2,0,0,NaN,209912
1,ZYGR2015103,0,1,1,2,1,0,NaN,209912


In [95]:
h_data['数据日期'] = h_data['数据日期'].astype('object')

In [96]:
h_data.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   2696 non-null   object 
 1   计价类型        2696 non-null   int64  
 2   周期类型        2696 non-null   int64  
 3   模式          2696 non-null   int64  
 4   风险等级        2696 non-null   int64  
 5   是否允许变更分红方式  2696 non-null   int64  
 6   产品品种        2696 non-null   int64  
 7   模式2         65 non-null     float64
 8   数据日期        2696 non-null   object 
dtypes: float64(1), int64(6), object(2)
memory usage: 189.7+ KB


In [97]:
h_data.drop(['产品品种', '模式2'], axis=1, inplace=True)

### B产品表2

In [98]:
l_data = pd.read_csv(path2 + 'l.csv')
l_data.head(2)

,prod_code,l1,l2,l3,l4,l5,l6,l7
0,YXFB2017031,0.0,2.0,3.0,1,NaN,0,209912
1,ZYGR2016014,1.0,2.0,1.0,2,4.0,1,209912


In [99]:
l_columns = ['prod_code', '募集方式', '管理方式', '业务模式', '收益特点', '期限', '投资模式', '数据日期']

In [100]:
l_data.columns = l_columns
l_data.head(2)

,prod_code,募集方式,管理方式,业务模式,收益特点,期限,投资模式,数据日期
0,YXFB2017031,0.0,2.0,3.0,1,NaN,0,209912
1,ZYGR2016014,1.0,2.0,1.0,2,4.0,1,209912


In [101]:
l_data['数据日期'] = l_data['数据日期'].astype('object')

In [102]:
l_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prod_code  3002 non-null   object 
 1   募集方式       2502 non-null   float64
 2   管理方式       2502 non-null   float64
 3   业务模式       2502 non-null   float64
 4   收益特点       3002 non-null   int64  
 5   期限         567 non-null    float64
 6   投资模式       3002 non-null   int64  
 7   数据日期       3002 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 187.8+ KB


### B产品总表

In [103]:
b_prod = pd.merge(h_data, l_data, on=['prod_code', '数据日期'], how='outer')
b_prod['prod_class'] = 3
b_prod.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,prod_class
0,ZYGR2016286,0.0,1.0,1.0,2.0,0.0,209912.0,0.0,2.0,1.0,2,NaN,1,3
1,ZYGR2015103,0.0,1.0,1.0,2.0,1.0,209912.0,1.0,2.0,1.0,2,6.0,1,3
2,YQ2017167,0.0,1.0,1.0,1.0,0.0,209912.0,0.0,2.0,1.0,2,NaN,0,3


In [104]:
b_prod['计价类型'].fillna(0, inplace=True)
b_prod['周期类型'].fillna(1, inplace=True)
b_prod['模式'].fillna(1, inplace=True)
b_prod['风险等级'].fillna(3, inplace=True)
b_prod['是否允许变更分红方式'].fillna(0, inplace=True)

In [105]:
b_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 0 to 3001
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   3002 non-null   object 
 1   计价类型        3002 non-null   float64
 2   周期类型        3002 non-null   float64
 3   模式          3002 non-null   float64
 4   风险等级        3002 non-null   float64
 5   是否允许变更分红方式  3002 non-null   float64
 6   数据日期        3002 non-null   float64
 7   募集方式        2502 non-null   float64
 8   管理方式        2502 non-null   float64
 9   业务模式        2502 non-null   float64
 10  收益特点        3002 non-null   int64  
 11  期限          567 non-null    float64
 12  投资模式        3002 non-null   int64  
 13  prod_class  3002 non-null   int64  
dtypes: float64(10), int64(3), object(1)
memory usage: 351.8+ KB


In [106]:
b_prod.nunique()

prod_code     3002
计价类型             2
周期类型             2
模式               2
风险等级             3
是否允许变更分红方式       2
数据日期             3
募集方式             2
管理方式             2
业务模式             2
收益特点             2
期限               5
投资模式             2
prod_class       1
dtype: int64

In [107]:
b_prod.describe()

,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,prod_class
count,3002.000000,3002.000000,3002.000000,3002.000000,3002.000000,3002.000000,2502.000000,2502.000000,2502.000000,3002.000000,567.000000,3002.000000,3002.0
mean,0.021652,0.992672,1.007328,1.955030,0.096269,208626.928714,0.225020,1.955635,1.252598,1.799134,4.716049,0.613258,3.0
std,0.145569,0.085306,0.085306,0.651793,0.295009,2960.826237,0.417679,0.205945,0.664505,0.400715,0.714914,0.487085,0.0
min,0.000000,0.000000,1.000000,1.000000,0.000000,201807.000000,0.000000,1.000000,1.000000,1.000000,3.000000,0.000000,3.0
25%,0.000000,1.000000,1.000000,2.000000,0.000000,209912.000000,0.000000,2.000000,1.000000,2.000000,4.000000,0.000000,3.0
50%,0.000000,1.000000,1.000000,2.000000,0.000000,209912.000000,0.000000,2.000000,1.000000,2.000000,5.000000,1.000000,3.0
75%,0.000000,1.000000,1.000000,2.000000,0.000000,209912.000000,0.000000,2.000000,1.000000,2.000000,5.000000,1.000000,3.0
max,1.000000,1.000000,2.000000,3.000000,1.000000,209912.000000,1.000000,2.000000,3.000000,2.000000,7.000000,1.000000,3.0


### C产品表1

In [108]:
i_data = pd.read_csv(path2 + 'i.csv')
i_data.head(2)

,prod_code,i1,i2,i3,i4,i5,i6,i7,i8,i9
0,DECD21052405,0,0,2,NaN,0,0,0,0,209912
1,DECD21110407,0,0,2,NaN,0,0,0,0,209912


In [109]:
i_columns = ['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '份额冻结比率', '产品品种', '持有天数', '数据日期']

In [110]:
i_data.columns = i_columns
i_data.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,份额冻结比率,产品品种,持有天数,数据日期
0,DECD21052405,0,0,2,NaN,0,0,0,0,209912
1,DECD21110407,0,0,2,NaN,0,0,0,0,209912


In [111]:
i_data['数据日期'] = i_data['数据日期'].astype('object')

In [112]:
i_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   129 non-null    object 
 1   计价类型        129 non-null    int64  
 2   周期类型        129 non-null    int64  
 3   模式          129 non-null    int64  
 4   风险等级        0 non-null      float64
 5   是否允许变更分红方式  129 non-null    int64  
 6   份额冻结比率      129 non-null    int64  
 7   产品品种        129 non-null    int64  
 8   持有天数        129 non-null    int64  
 9   数据日期        129 non-null    object 
dtypes: float64(1), int64(7), object(2)
memory usage: 10.2+ KB


In [113]:
i_data.drop(['份额冻结比率', '产品品种'], axis=1, inplace=True)

### C产品表2

In [114]:
m_data = pd.read_csv(path2 + 'm.csv')
m_data.head(2)

,prod_code,m1,m2,m3,m4,m5,m6,m7,m8,m9
0,DECD21110407,NaN,NaN,NaN,NaN,NaN,2,0.038375,5.0,209912
1,DECD21110101,NaN,NaN,NaN,NaN,NaN,2,0.043625,1.0,209912


In [115]:
m_columns = ['prod_code', '募集方式', '管理方式', '业务模式', '收益特点', '期限', '投资模式', '预期收益率', '展示等级', '数据日期']

In [116]:
m_data.columns = m_columns
m_data.head(2)

,prod_code,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,数据日期
0,DECD21110407,NaN,NaN,NaN,NaN,NaN,2,0.038375,5.0,209912
1,DECD21110101,NaN,NaN,NaN,NaN,NaN,2,0.043625,1.0,209912


In [117]:
m_data['数据日期'] = m_data['数据日期'].astype('object')

In [118]:
m_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prod_code  129 non-null    object 
 1   募集方式       0 non-null      float64
 2   管理方式       0 non-null      float64
 3   业务模式       0 non-null      float64
 4   收益特点       0 non-null      float64
 5   期限         0 non-null      float64
 6   投资模式       129 non-null    int64  
 7   预期收益率      129 non-null    float64
 8   展示等级       120 non-null    float64
 9   数据日期       129 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 10.2+ KB


### C 产品总表

In [119]:
c_prod = pd.merge(i_data, m_data, on=['prod_code', '数据日期'], how='outer')
c_prod['prod_class'] = 2
c_prod.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class
0,DECD21052405,0,0,2,NaN,0,0,209912,NaN,NaN,NaN,NaN,NaN,2,0.035278,NaN,2
1,DECD21110407,0,0,2,NaN,0,0,209912,NaN,NaN,NaN,NaN,NaN,2,0.038375,5.0,2


In [120]:
c_prod['展示等级'].fillna(5, inplace=True)

In [121]:
c_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 128
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   129 non-null    object 
 1   计价类型        129 non-null    int64  
 2   周期类型        129 non-null    int64  
 3   模式          129 non-null    int64  
 4   风险等级        0 non-null      float64
 5   是否允许变更分红方式  129 non-null    int64  
 6   持有天数        129 non-null    int64  
 7   数据日期        129 non-null    object 
 8   募集方式        0 non-null      float64
 9   管理方式        0 non-null      float64
 10  业务模式        0 non-null      float64
 11  收益特点        0 non-null      float64
 12  期限          0 non-null      float64
 13  投资模式        129 non-null    int64  
 14  预期收益率       129 non-null    float64
 15  展示等级        129 non-null    float64
 16  prod_class  129 non-null    int64  
dtypes: float64(8), int64(7), object(2)
memory usage: 18.1+ KB


In [122]:
c_prod.nunique()

prod_code     129
计价类型            2
周期类型            2
模式              2
风险等级            0
是否允许变更分红方式      1
持有天数            1
数据日期            1
募集方式            0
管理方式            0
业务模式            0
收益特点            0
期限              0
投资模式            1
预期收益率          12
展示等级            2
prod_class      1
dtype: int64

In [123]:
c_prod.describe()

,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,持有天数,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class
count,129.000000,129.000000,129.000000,0.0,129.0,129.0,0.0,0.0,0.0,0.0,0.0,129.0,129.000000,129.000000,129.0
mean,0.015504,0.015504,1.984496,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.037139,4.658915,2.0
std,0.124027,0.124027,0.124027,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.005414,1.121495,0.0
min,0.000000,0.000000,1.000000,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.019002,1.000000,2.0
25%,0.000000,0.000000,2.000000,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.038375,5.000000,2.0
50%,0.000000,0.000000,2.000000,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.038375,5.000000,2.0
75%,0.000000,0.000000,2.000000,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.038375,5.000000,2.0
max,1.000000,1.000000,2.000000,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.045856,5.000000,2.0


### D产品表

In [124]:
j_data = pd.read_csv(path2 + 'j.csv')
j_data.head(2)

,prod_code,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11,j12,j13
0,FYG21040C,1,1,6.0,2,1,2,0.0011,0.0011,1,1,4.4735,1,202112
1,H201224A,1,1,6.0,1,0,1,0.0368,0.0368,1,1,NaN,1,202112


In [125]:
j_columns = ['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '收益计息基数', '最低收益率', '最高收益率', 
             '产品面值', '发行价格', '预期收益率', '持有天数', '数据日期']

In [126]:
j_data.columns = j_columns
j_data.head(2)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,收益计息基数,最低收益率,最高收益率,产品面值,发行价格,预期收益率,持有天数,数据日期
0,FYG21040C,1,1,6.0,2,1,2,0.0011,0.0011,1,1,4.4735,1,202112
1,H201224A,1,1,6.0,1,0,1,0.0368,0.0368,1,1,NaN,1,202112


In [127]:
j_data['数据日期'] = j_data['数据日期'].astype('object')

In [128]:
j_data.drop(['收益计息基数', '最低收益率', '最高收益率', '产品面值', '发行价格'], axis=1, inplace=True)

In [129]:
j_data['模式'].fillna(6, inplace=True)
j_data['预期收益率'].fillna(4.1, inplace=True)

In [130]:
d_prod = j_data.copy()

In [131]:
d_prod['prod_class'] = 4

In [132]:
d_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   231 non-null    object 
 1   计价类型        231 non-null    int64  
 2   周期类型        231 non-null    int64  
 3   模式          231 non-null    float64
 4   风险等级        231 non-null    int64  
 5   是否允许变更分红方式  231 non-null    int64  
 6   预期收益率       231 non-null    float64
 7   持有天数        231 non-null    int64  
 8   数据日期        231 non-null    object 
 9   prod_class  231 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 18.2+ KB


In [133]:
d_prod.nunique()

prod_code     231
计价类型            1
周期类型            2
模式              3
风险等级            3
是否允许变更分红方式      2
预期收益率          32
持有天数            1
数据日期            1
prod_class      1
dtype: int64

In [134]:
d_prod.describe()

,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,预期收益率,持有天数,prod_class
count,231.0,231.000000,231.000000,231.000000,231.000000,231.000000,231.0,231.0
mean,1.0,0.917749,5.861472,1.839827,0.225108,4.101308,1.0,4.0
std,0.0,0.275343,0.744741,0.682303,0.418561,0.525355,0.0,0.0
min,1.0,0.000000,1.000000,1.000000,0.000000,0.116000,1.0,4.0
25%,1.0,1.000000,6.000000,1.000000,0.000000,4.027250,1.0,4.0
50%,1.0,1.000000,6.000000,2.000000,0.000000,4.100000,1.0,4.0
75%,1.0,1.000000,6.000000,2.000000,0.000000,4.211000,1.0,4.0
max,1.0,1.000000,6.000000,3.000000,1.000000,5.103500,1.0,4.0


In [135]:
a_prod.columns

Index(['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '持有天数', '数据日期',
       '募集方式', '管理方式', '业务模式', '收益特点', '期限', '投资模式', '预期收益率', '展示等级',
       'prod_class'],
      dtype='object')

In [136]:
b_prod.columns

Index(['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '数据日期', '募集方式',
       '管理方式', '业务模式', '收益特点', '期限', '投资模式', 'prod_class'],
      dtype='object')

In [137]:
c_prod.columns

Index(['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '持有天数', '数据日期',
       '募集方式', '管理方式', '业务模式', '收益特点', '期限', '投资模式', '预期收益率', '展示等级',
       'prod_class'],
      dtype='object')

In [138]:
d_prod.columns

Index(['prod_code', '计价类型', '周期类型', '模式', '风险等级', '是否允许变更分红方式', '预期收益率',
       '持有天数', '数据日期', 'prod_class'],
      dtype='object')

### A+B+C+D的产品表

In [139]:
prod_data = pd.concat([a_prod, b_prod, c_prod, d_prod])
prod_data.reset_index(inplace=True)
del prod_data['index']
prod_data.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class
0,ZXGRHLA2021059,0.0,1.0,1.0,1.0,0.0,364.0,209912.0,0.0,2.0,1.0,2.0,NaN,1.0,0.042680,5.0,1
1,GRSHB201902005,0.0,1.0,1.0,1.0,0.0,1.0,209912.0,0.0,2.0,1.0,2.0,NaN,1.0,0.027350,5.0,1
2,GRHLA20211629,0.0,1.0,1.0,1.0,0.0,112.0,209912.0,0.0,2.0,1.0,2.0,5.0,1.0,0.040475,2.0,1


In [140]:
prod_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26266 entries, 0 to 26265
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   prod_code   26266 non-null  object 
 1   计价类型        26266 non-null  float64
 2   周期类型        26266 non-null  float64
 3   模式          26266 non-null  float64
 4   风险等级        26137 non-null  float64
 5   是否允许变更分红方式  26266 non-null  float64
 6   持有天数        6917 non-null   float64
 7   数据日期        26266 non-null  object 
 8   募集方式        25406 non-null  float64
 9   管理方式        25406 non-null  float64
 10  业务模式        25406 non-null  float64
 11  收益特点        25876 non-null  float64
 12  期限          3152 non-null   float64
 13  投资模式        26035 non-null  float64
 14  预期收益率       23234 non-null  float64
 15  展示等级        23033 non-null  float64
 16  prod_class  26266 non-null  int64  
dtypes: float64(14), int64(1), object(2)
memory usage: 3.4+ MB


In [141]:
prod_data.nunique()

prod_code     25825
计价类型              2
周期类型              2
模式                4
风险等级              3
是否允许变更分红方式        2
持有天数            320
数据日期             32
募集方式              2
管理方式              2
业务模式              2
收益特点              2
期限                6
投资模式              3
预期收益率           250
展示等级              5
prod_class        4
dtype: int64

In [142]:
prod_data.to_pickle('product.pkl')

### A产品流水表

In [143]:
n_data = pd.read_csv(path2 + 'n.csv', thousands=',')
#n_data = reduce_mem_usage(n_data)
n_data.head(2)

,n1,n2,n3,core_cust_id,prod_code,n6,n7,n8,n9,n10,n11
0,JGX202101020000005963478,4.0,1,48e046e615,GRHLA2020552,1,220610.0,1,2,0,20210102
1,JGX202101200000006096306,8.0,5,48e046e615,GRSHC2020390,1,110.0,6,3,0,20210120


In [144]:
n_columns = ['流水号', '业务代码', '渠道标识', 'core_cust_id', 'prod_code', '净值', '申请金额', '资金状态', '交易状态', '总金额', 'trade_date']

In [145]:
n_data.columns = n_columns

In [146]:
n_data['trade_date'] = n_data['trade_date'].astype('str')
#n_data['trade_date'] = pd.to_datetime(n_data['trade_date'], format='%Y%m%d', errors='coerce')
n_data['prod_class'] = 1
n_data.head(2)

,流水号,业务代码,渠道标识,core_cust_id,prod_code,净值,申请金额,资金状态,交易状态,总金额,trade_date,prod_class
0,JGX202101020000005963478,4.0,1,48e046e615,GRHLA2020552,1,220610.0,1,2,0,20210102,1
1,JGX202101200000006096306,8.0,5,48e046e615,GRSHC2020390,1,110.0,6,3,0,20210120,1


In [147]:
n_data.sort_values(by=['core_cust_id', 'prod_code'], inplace=True)
n_data['业务代码'] = n_data['业务代码'].fillna(method='ffill')

In [148]:
n_data.describe()

,业务代码,渠道标识,净值,申请金额,资金状态,交易状态,总金额,prod_class
count,949114.000000,949114.000000,949114.0,9.491140e+05,949114.000000,949114.000000,949114.0,949114.0
mean,6.867582,4.024537,1.0,1.081216e+05,3.452229,3.115492,0.0,1.0
std,2.313045,1.689281,0.0,2.434524e+06,2.352625,0.722830,0.0,0.0
min,4.000000,0.000000,1.0,1.100000e+02,0.000000,1.000000,0.0,1.0
25%,4.000000,2.000000,1.0,1.100000e+02,3.000000,3.000000,0.0,1.0
50%,7.000000,5.000000,1.0,1.100000e+02,3.000000,3.000000,0.0,1.0
75%,8.000000,5.000000,1.0,7.610000e+03,6.000000,3.000000,0.0,1.0
max,10.000000,5.000000,1.0,5.250001e+08,8.000000,7.000000,0.0,1.0


### B产品流水表

In [149]:
o_data = pd.read_csv(path2 + 'o.csv', thousands=',')
o_data.head(2)

,o1,o2,o3,core_cust_id,prod_code,o6,o7,o8,o9,o10,o11,o12
0,ALC202107210000007437420,2,1,b1a66424c4,SSTJMZKF002,1.0085,75110.0,3,3,110.0,110.0,20210721
1,ALC202107140000007373365,2,1,e733784b55,SSTJMZKF002,1.0071,18110.0,3,3,110.0,110.0,20210714


In [150]:
o_columns = ['流水号', '业务代码', '渠道标识', 'core_cust_id', 'prod_code', '净值', '申请金额', 
             '交易状态', '资金状态', '总金额', '超额管理费', 'trade_date']

In [151]:
o_data.columns = o_columns
o_data['prod_class'] = 3
o_data['trade_date'] = o_data['trade_date'].astype('str')
#o_data['trade_date'] = pd.to_datetime(o_data['trade_date'], format='%Y%m%d', errors='coerce')
o_data.head(2)

,流水号,业务代码,渠道标识,core_cust_id,prod_code,净值,申请金额,交易状态,资金状态,总金额,超额管理费,trade_date,prod_class
0,ALC202107210000007437420,2,1,b1a66424c4,SSTJMZKF002,1.0085,75110.0,3,3,110.0,110.0,20210721,3
1,ALC202107140000007373365,2,1,e733784b55,SSTJMZKF002,1.0071,18110.0,3,3,110.0,110.0,20210714,3


In [152]:
o_data['净值'] = o_data['净值'].apply(lambda x: 1.0 if x<0 else x)

In [153]:
#净值里面异常值改为1.0,超额管理费要删除么？
o_data.describe()

,业务代码,渠道标识,净值,申请金额,交易状态,资金状态,总金额,超额管理费,prod_class
count,42492.000000,42492.000000,42492.000000,4.249200e+04,42492.000000,42492.000000,4.249200e+04,42492.000000,42492.0
mean,4.264003,2.224772,1.003571,1.323012e+05,3.181446,3.655417,1.773845e+04,116.756094,3.0
std,2.925424,1.799264,0.052764,7.725152e+05,0.932282,2.074979,1.608551e+05,66.730652,0.0
min,2.000000,0.000000,0.000000,1.100000e+02,1.000000,0.000000,1.100000e+02,110.000000,3.0
25%,2.000000,1.000000,1.000000,1.100000e+02,3.000000,3.000000,1.100000e+02,110.000000,3.0
50%,2.000000,1.000000,1.004100,3.011000e+04,3.000000,3.000000,1.100000e+02,110.000000,3.0
75%,8.000000,5.000000,1.011700,1.291100e+05,3.000000,6.000000,1.100000e+02,110.000000,3.0
max,11.000000,5.000000,1.024600,1.120801e+08,7.000000,8.000000,1.643143e+07,6242.630000,3.0


### C产品流水表

In [154]:
q_data = pd.read_csv(path2 + 'q.csv', thousands=',')
#q_data = reduce_mem_usage(q_data)
q_data.head(2)

,q1,q2,q3,core_cust_id,prod_code,q6,q7,q8,q9,q10
0,ADE202109010000007727221,2,0,9cb205e4c9,DECD21090108,1,1005110,3,3,20210901
1,ADE202107060000007312643,2,2,3f9024560b,DECD21062101,1,312110,3,3,20210706


In [155]:
q_columns = ['流水号', '业务代码', '渠道标识', 'core_cust_id', 'prod_code', '净值', '申请金额', '资金状态', '交易状态', 'trade_date']

In [156]:
q_data.columns = q_columns
q_data['prod_class'] = 2
q_data['trade_date'] = q_data['trade_date'].astype('str')
#q_data['trade_date'] = pd.to_datetime(q_data['trade_date'], format='%Y%m%d', errors='coerce')
q_data.head(2)

,流水号,业务代码,渠道标识,core_cust_id,prod_code,净值,申请金额,资金状态,交易状态,trade_date,prod_class
0,ADE202109010000007727221,2,0,9cb205e4c9,DECD21090108,1,1005110,3,3,20210901,2
1,ADE202107060000007312643,2,2,3f9024560b,DECD21062101,1,312110,3,3,20210706,2


In [157]:
q_data.describe()

,业务代码,渠道标识,净值,申请金额,资金状态,交易状态,prod_class
count,3886.000000,3886.000000,3886.0,3.886000e+03,3886.000000,3886.000000,3886.0
mean,2.003088,1.848173,1.0,5.537490e+05,3.001544,3.001544,2.0
std,0.078537,0.529788,0.0,2.796352e+06,0.039269,0.039269,0.0
min,2.000000,0.000000,1.0,3.001100e+05,3.000000,3.000000,2.0
25%,2.000000,2.000000,1.0,3.001100e+05,3.000000,3.000000,2.0
50%,2.000000,2.000000,1.0,3.001100e+05,3.000000,3.000000,2.0
75%,2.000000,2.000000,1.0,4.501100e+05,3.000000,3.000000,2.0
max,4.000000,2.000000,1.0,7.500011e+07,4.000000,4.000000,2.0


### D产品流水表

In [158]:
p_data = pd.read_csv(path2 + 'p.csv', thousands=',')
#p_data = reduce_mem_usage(p_data)
p_data.head(2)

,p1,p2,p3,core_cust_id,prod_code,p6,p7,p8,p9,p10,p11,p12
0,DLC202101100000000013522,9,1.0,bf9eaccd54,H210107M,1.169712,75110.0,0.0011,3,3.0,0,20210110
1,DLC202101100000000013521,4,1.0,bf9eaccd54,H210107M,1.169712,75110.0,0.0011,2,3.0,0,20210110


In [159]:
p_columns = ['流水号', '业务代码', '渠道标识', 'core_cust_id', 'prod_code', '净值', '申请金额', '折扣率', '交易状态', '资金状态', '费率', 'trade_date']

In [160]:
p_data.columns = p_columns
p_data['prod_class'] = 4
p_data['trade_date'] = p_data['trade_date'].astype('str')
#p_data['trade_date'] = pd.to_datetime(p_data['trade_date'], format='%Y%m%d', errors='coerce')
p_data.head(2)

,流水号,业务代码,渠道标识,core_cust_id,prod_code,净值,申请金额,折扣率,交易状态,资金状态,费率,trade_date,prod_class
0,DLC202101100000000013522,9,1.0,bf9eaccd54,H210107M,1.169712,75110.0,0.0011,3,3.0,0,20210110,4
1,DLC202101100000000013521,4,1.0,bf9eaccd54,H210107M,1.169712,75110.0,0.0011,2,3.0,0,20210110,4


In [161]:
#按照core_cust_id和prod_code排序，向上填充
p_data.sort_values(by=['core_cust_id', 'prod_code'], inplace=True)
p_data['渠道标识'] = p_data['渠道标识'].fillna(method='ffill')

In [162]:
#按照core_cust_id和prod_code排序，向上填充
p_data.sort_values(by=['core_cust_id', 'prod_code'], inplace=True)
p_data['资金状态'] = p_data['资金状态'].fillna(method='ffill')

In [163]:
p_data['净值'] = p_data['净值'].apply(lambda x:round(x, 6))

In [164]:
p_data.describe()

,业务代码,渠道标识,净值,申请金额,折扣率,交易状态,资金状态,费率,prod_class
count,1.056586e+06,1.056586e+06,1.056586e+06,1.056586e+06,1.056586e+06,1.056586e+06,1.056586e+06,1.056586e+06,1056586.0
mean,9.094583e+00,1.179598e+00,1.162081e+00,2.499610e+04,5.680630e-04,3.038944e+00,2.190266e+00,1.516204e-03,4.0
std,2.414170e+00,7.792072e-01,2.625669e-02,5.166628e+05,1.864037e-02,4.123497e-01,1.498176e+00,3.890895e-02,0.0
min,2.000000e+00,1.000000e+00,1.100000e-01,1.100000e+02,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,4.0
25%,1.000000e+01,1.000000e+00,1.160000e+00,1.103750e+02,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,4.0
50%,1.000000e+01,1.000000e+00,1.160000e+00,1.125350e+02,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,4.0
75%,1.000000e+01,1.000000e+00,1.160000e+00,1.263800e+02,0.000000e+00,3.000000e+00,4.000000e+00,0.000000e+00,4.0
max,1.000000e+01,6.000000e+00,1.299293e+00,1.650001e+08,1.000000e+00,7.000000e+00,5.000000e+00,1.000000e+00,4.0


### 流水表汇总

In [165]:
record = pd.concat([n_data, o_data, q_data, p_data], ignore_index=True)

In [166]:
record.drop(['总金额', '超额管理费', '折扣率', '费率'], axis=1, inplace=True)

In [167]:
record.head(3)

,流水号,业务代码,渠道标识,core_cust_id,prod_code,净值,申请金额,资金状态,交易状态,trade_date,prod_class
0,JGX202103110000006437222,10.0,5.0,1087ee0a95,GRSHA2020255,1.0,110.0,0.0,3,20210311,1
1,JGX202103110000006440647,7.0,5.0,1087ee0a95,GRSHA2020255,1.0,110.0,6.0,3,20210311,1
2,JGX202103110000006439605,8.0,5.0,1087ee0a95,GRSHA2020255,1.0,110.0,6.0,3,20210311,1


In [168]:
record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2052078 entries, 0 to 2052077
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   流水号           object 
 1   业务代码          float64
 2   渠道标识          float64
 3   core_cust_id  object 
 4   prod_code     object 
 5   净值            float64
 6   申请金额          float64
 7   资金状态          float64
 8   交易状态          int64  
 9   trade_date    object 
 10  prod_class    int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 172.2+ MB


In [169]:
record.nunique()

流水号             2052078
业务代码                 10
渠道标识                  5
core_cust_id      62074
prod_code          2810
净值                  331
申请金额              39122
资金状态                  8
交易状态                  5
trade_date          273
prod_class            4
dtype: int64

In [170]:
record.isnull().sum()

流水号             0
业务代码            0
渠道标识            0
core_cust_id    0
prod_code       0
净值              0
申请金额            0
资金状态            0
交易状态            0
trade_date      0
prod_class      0
dtype: int64

In [171]:
record.to_pickle('prod_record.pkl')

In [172]:
record['交易状态'].value_counts()

3    1988446
7      44284
2      18875
4        388
1         85
Name: 交易状态, dtype: int64

In [255]:
#record[record['core_cust_id'] == '1087ee0a95']

#### 统计流水总数

按照用户和产品编号聚类
- 统计6、7、8、9月份流水总数
- 统计7.1前，8.1日前，9.1日前，10.1日前的流水总数

In [348]:
record_6_7 = record[(record['trade_date']>'2021-05-31') & (record['trade_date']<'2021-07-01')]
record_7_8 = record[(record['trade_date']>'2021-06-30') & (record['trade_date']<'2021-08-01')]
record_8_9 = record[(record['trade_date']>'2021-07-31') & (record['trade_date']<'2021-09-01')]
record_9_10 = record[(record['trade_date']>'2021-08-31') & (record['trade_date']<'2021-10-01')]

In [349]:
record_b_7 = record[record['trade_date']<'2021-07-01']
record_b_8 = record[record['trade_date']<'2021-08-01']
record_b_9 = record[record['trade_date']<'2021-09-01']
record_b_10 = record.copy()

以往流水总数

In [ ]:
record_6_7['rw_6_7'] = record_6_7.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_7_8['rw_7_8'] = record_7_8.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_8_9['rw_8_9'] = record_8_9.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_9_10['rw_9_10'] = record_9_10.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()

In [ ]:
record_b_7['rw_b_7'] = record_b_7.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_b_8['rw_b_8'] = record_b_8.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_b_9['rw_b_9'] = record_b_9.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()
record_b_10['rw_b_10'] = record_b_10.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()

交易日期最早

In [ ]:
record_b_7['date_b_7'] = record_b_7.groupby(['core_cust_id', 'prod_code'])['trade_date'].transform(lambda x:x.min()).values.tolist()
record_b_8['date_b_8'] = record_b_8.groupby(['core_cust_id', 'prod_code'])['trade_date'].transform(lambda x:x.min()).values.tolist()
record_b_9['date_b_9'] = record_b_9.groupby(['core_cust_id', 'prod_code'])['trade_date'].transform(lambda x:x.min()).values.tolist()
record_b_10['date_b_10'] = record_b_10.groupby(['core_cust_id', 'prod_code'])['trade_date'].transform(lambda x:x.min()).values.tolist()

交易状态为3表示成功

In [350]:
record_6_7['su_6_7'] = record_6_7.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:(x==3).count()).values.tolist()
record_7_8['su_7_8'] = record_7_8.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:(x==3).count()).values.tolist()
record_8_9['su_8_9'] = record_8_9.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:(x==3).count()).values.tolist()
record_9_10['su_9_10'] = record_9_10.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:(x==3).count()).values.tolist()

In [ ]:
record_b_7['su_b_7'] = record_b_7.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:(x==3).count()).values.tolist()
record_b_8['su_b_8'] = record_b_8.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:x.count()).values.tolist()
record_b_9['su_b_9'] = record_b_9.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:x.count()).values.tolist()
record_b_10['su_b_10'] = record_b_10.groupby(['core_cust_id', 'prod_code'])['交易状态'].transform(lambda x:x.count()).values.tolist()

In [257]:
# 7.1前
record['before_7'] = record.groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()

ValueError: Length of values (1035389) does not match length of index (2052078)

In [ ]:
#8.1前
record['before_7'] = record[record['trade_date']<'2021-07-01'].groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()

In [ ]:
#9.1前
record['before_7'] = record[record['trade_date']<'2021-07-01'].groupby(['core_cust_id', 'prod_code'])['流水号'].transform(lambda x:x.count()).values.tolist()

### 与train和test做对比

#### 产品对比

In [231]:
prod_4 = record['prod_code'].unique().tolist()

In [233]:
len(prod_4), len(using_prod)

(2810, 129)

In [234]:
#prod_4:2810, using_prod:129
len(set(prod_4) & using_prod)

118

- 所有训练预测产品均在ABCD产品表中

#### 用户对比

In [235]:
cust_4 = record['core_cust_id'].unique().tolist()

In [236]:
len(cust_4), len(using_cust), len(buy_cust)

(62074, 264055, 16656)

In [237]:
len(set(cust_4) & set(buy_cust))

16656

In [239]:
len(set(cust_4) & set(train_cust))

58592

In [238]:
#cust_4:62074, using_cust:264055
len(set(cust_4) & using_cust)

59372

- 训练测试的用户大部分都没有在ABCD流水表中，且一部分用户流水没有出现在训练测试集中

### A产品基本信息+流水表

In [133]:
A_df = pd.merge(a_prod, n_data, on=['prod_code', 'prod_class'], how='outer')

In [134]:
A_df.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否滚存,是否允许变更分红方式,产品品种,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,最高收益率,最低收益率,展示等级,prod_class,流水号,业务代码,渠道标识,core_cust_id,净值,申请金额,资金状态,交易状态,总金额,trade_date
0,ZXGRHLA2021059,0.0,1.0,1.0,1.0,0.0,0.0,0.0,364.0,2099-12-01,NaN,NaN,NaN,2.0,NaN,1.0,0.042694,0.043732,0.015793,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,GRSHB201902005,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2099-12-01,NaN,NaN,NaN,2.0,NaN,1.0,0.027344,0.027344,0.001100,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,GRHLA20211629,0.0,1.0,1.0,1.0,0.0,0.0,0.0,112.0,2099-12-01,NaN,NaN,NaN,2.0,5.0,1.0,0.040466,0.043091,0.015793,2.0,1,JGX202109260000007886008,4.0,1.0,2847abcb4d,1.0,"60,110",3.0,3.0,0.0,2021-09-26


In [135]:
A_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 969443 entries, 0 to 969442
Data columns (total 31 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   prod_code     969443 non-null  object        
 1   计价类型          953096 non-null  float64       
 2   周期类型          953096 non-null  float64       
 3   模式            953096 non-null  float64       
 4   风险等级          953096 non-null  float64       
 5   是否滚存          953096 non-null  float64       
 6   是否允许变更分红方式    953096 non-null  float64       
 7   产品品种          953096 non-null  float64       
 8   持有天数          953096 non-null  float64       
 9   数据日期          969443 non-null  datetime64[ns]
 10  募集方式          314 non-null     float16       
 11  管理方式          314 non-null     float16       
 12  业务模式          314 non-null     float16       
 13  收益特点          969413 non-null  float64       
 14  期限            416667 non-null  float16       
 15  投资模式          969

In [136]:
A_df.nunique()

prod_code        22899
计价类型                 1
周期类型                 1
模式                   1
风险等级                 2
是否滚存                 2
是否允许变更分红方式           1
产品品种                 1
持有天数               319
数据日期                30
募集方式                 1
管理方式                 1
业务模式                 1
收益特点                 1
期限                   6
投资模式                 1
预期收益率              213
最高收益率              214
最低收益率               25
展示等级                 5
prod_class           1
流水号             949114
业务代码                 7
渠道标识                 4
core_cust_id     51888
净值                   1
申请金额              2155
资金状态                 6
交易状态                 4
总金额                  1
trade_date         273
dtype: int64

### B产品基本信息+流水表

In [137]:
B_df = pd.merge(b_prod, o_data, on=['prod_code', 'prod_class'], how='outer')

In [138]:
B_df.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,产品品种,模式2,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,prod_class,流水号,业务代码,渠道标识,core_cust_id,净值,申请金额,交易状态,资金状态,总金额,超额管理费,trade_date
0,ZYGR2016286,0.0,1.0,1.0,2.0,0.0,0.0,NaN,2099-12-01,0.0,2.0,1.0,2,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,ZYGR2015103,0.0,1.0,1.0,2.0,1.0,0.0,NaN,2099-12-01,1.0,2.0,1.0,2,6.0,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,YQ2017167,0.0,1.0,1.0,1.0,0.0,0.0,NaN,2099-12-01,0.0,2.0,1.0,2,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [139]:
B_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45382 entries, 0 to 45381
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   prod_code     45382 non-null  object        
 1   计价类型          45076 non-null  float64       
 2   周期类型          45076 non-null  float64       
 3   模式            45076 non-null  float64       
 4   风险等级          45076 non-null  float64       
 5   是否允许变更分红方式    45076 non-null  float64       
 6   产品品种          45076 non-null  float64       
 7   模式2           31017 non-null  float16       
 8   数据日期          45382 non-null  datetime64[ns]
 9   募集方式          36879 non-null  float16       
 10  管理方式          36879 non-null  float16       
 11  业务模式          36879 non-null  float16       
 12  收益特点          45382 non-null  int8          
 13  期限            8776 non-null   float16       
 14  投资模式          45382 non-null  int8          
 15  prod_class    45382 non-null  int64 

In [140]:
B_df.nunique()

prod_code        3002
计价类型                2
周期类型                2
模式                  2
风险等级                3
是否允许变更分红方式          2
产品品种                1
模式2                 7
数据日期                3
募集方式                2
管理方式                2
业务模式                2
收益特点                2
期限                  5
投资模式                2
prod_class          1
流水号             42492
业务代码                9
渠道标识                4
core_cust_id    13678
净值                 87
申请金额              741
交易状态                5
资金状态                5
总金额              1123
超额管理费             960
trade_date        188
dtype: int64

### C产品基本信息+流水表

In [141]:
C_df = pd.merge(c_prod, q_data, on=['prod_code', 'prod_class'], how='outer')

In [142]:
C_df.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,份额冻结比率,产品品种,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,展示等级,prod_class,流水号,业务代码,渠道标识,core_cust_id,净值,申请金额,资金状态,交易状态,trade_date
0,DECD21052405,0,0,2,NaN,0,0,0,0,2099-12-01,NaN,NaN,NaN,NaN,NaN,2,0.035278,NaN,2,ADE202106080000007065142,2.0,2.0,baf6949c96,1.0,"600,110",3.0,3.0,2021-06-08
1,DECD21052405,0,0,2,NaN,0,0,0,0,2099-12-01,NaN,NaN,NaN,NaN,NaN,2,0.035278,NaN,2,ADE202106090000007073728,2.0,2.0,d23f0d85df,1.0,"450,110",3.0,3.0,2021-06-09
2,DECD21052405,0,0,2,NaN,0,0,0,0,2099-12-01,NaN,NaN,NaN,NaN,NaN,2,0.035278,NaN,2,ADE202106100000007082691,2.0,2.0,7c195edce2,1.0,"300,110",3.0,3.0,2021-06-10


In [143]:
C_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3986 entries, 0 to 3985
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   prod_code     3986 non-null   object        
 1   计价类型          3986 non-null   int8          
 2   周期类型          3986 non-null   int8          
 3   模式            3986 non-null   int8          
 4   风险等级          0 non-null      float64       
 5   是否允许变更分红方式    3986 non-null   int8          
 6   份额冻结比率        3986 non-null   int8          
 7   产品品种          3986 non-null   int8          
 8   持有天数          3986 non-null   int8          
 9   数据日期          3986 non-null   datetime64[ns]
 10  募集方式          0 non-null      float64       
 11  管理方式          0 non-null      float64       
 12  业务模式          0 non-null      float64       
 13  收益特点          0 non-null      float64       
 14  期限            0 non-null      float64       
 15  投资模式          3986 non-null   int8    

In [144]:
C_df.nunique()

prod_code        129
计价类型               2
周期类型               2
模式                 2
风险等级               0
是否允许变更分红方式         1
份额冻结比率             1
产品品种               1
持有天数               1
数据日期               1
募集方式               0
管理方式               0
业务模式               0
收益特点               0
期限                 0
投资模式               1
预期收益率             12
展示等级               2
prod_class         1
流水号             3886
业务代码               2
渠道标识               2
core_cust_id    2852
净值                 1
申请金额             187
资金状态               2
交易状态               2
trade_date        84
dtype: int64

### D产品基本信息+流水表

In [145]:
D_df = pd.merge(d_prod, p_data, on=['prod_code', 'prod_class'], how='outer')

In [146]:
D_df.head(3)

,prod_code,计价类型,周期类型,模式,风险等级,是否允许变更分红方式,收益计息基数,最低收益率,最高收益率,产品面值,发行价格,预期收益率,持有天数,数据日期,prod_class,流水号,业务代码,渠道标识,core_cust_id,净值,申请金额,折扣率,交易状态,资金状态,费率,trade_date
0,FYG21040C,1,1,6.0,2,1,2,0.001100,0.001100,1,1,4.472656,1,2021-12-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,H201224A,1,1,6.0,1,0,1,0.036804,0.036804,1,1,NaN,1,2021-12-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,H201029B,1,1,6.0,1,0,1,0.038361,0.038361,1,1,NaN,1,2021-12-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [147]:
D_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056723 entries, 0 to 1056722
Data columns (total 26 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   prod_code     1056723 non-null  object        
 1   计价类型          1056723 non-null  int8          
 2   周期类型          1056723 non-null  int8          
 3   模式            1056716 non-null  float16       
 4   风险等级          1056723 non-null  int8          
 5   是否允许变更分红方式    1056723 non-null  int8          
 6   收益计息基数        1056723 non-null  int8          
 7   最低收益率         1056723 non-null  float16       
 8   最高收益率         1056723 non-null  float16       
 9   产品面值          1056723 non-null  int8          
 10  发行价格          1056723 non-null  int8          
 11  预期收益率         1056640 non-null  float16       
 12  持有天数          1056723 non-null  int8          
 13  数据日期          1056723 non-null  datetime64[ns]
 14  prod_class    1056723 non-null  int64         
 15

In [148]:
D_df.nunique()

prod_code           231
计价类型                  1
周期类型                  2
模式                    3
风险等级                  3
是否允许变更分红方式            2
收益计息基数                3
最低收益率                16
最高收益率                16
产品面值                  2
发行价格                  2
预期收益率                31
持有天数                  1
数据日期                  1
prod_class            1
流水号             1056586
业务代码                  6
渠道标识                  3
core_cust_id      19012
净值                   70
申请金额              37838
折扣率                   7
交易状态                  4
资金状态                  5
费率                    2
trade_date          273
dtype: int64

### A+B+C+D产品表

In [149]:
prod_df = pd.concat([A_df, B_df, C_df, D_df], ignore_index=True)

In [150]:
prod_df.head()

,prod_code,计价类型,周期类型,模式,风险等级,是否滚存,是否允许变更分红方式,产品品种,持有天数,数据日期,募集方式,管理方式,业务模式,收益特点,期限,投资模式,预期收益率,最高收益率,最低收益率,展示等级,prod_class,流水号,业务代码,渠道标识,core_cust_id,净值,申请金额,资金状态,交易状态,总金额,trade_date,模式2,超额管理费,份额冻结比率,收益计息基数,产品面值,发行价格,折扣率,费率
0,ZXGRHLA2021059,0.0,1.0,1.0,1.0,0.0,0.0,0.0,364.0,2099-12-01,NaN,NaN,NaN,2.0,NaN,1.0,0.042694,0.043732,0.015793,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GRSHB201902005,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2099-12-01,NaN,NaN,NaN,2.0,NaN,1.0,0.027344,0.027344,0.001100,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRHLA20211629,0.0,1.0,1.0,1.0,0.0,0.0,0.0,112.0,2099-12-01,NaN,NaN,NaN,2.0,5.0,1.0,0.040466,0.043091,0.015793,2.0,1,JGX202109260000007886008,4.0,1.0,2847abcb4d,1.0,"60,110",3.0,3.0,0.0,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GRHLA20211629,0.0,1.0,1.0,1.0,0.0,0.0,0.0,112.0,2099-12-01,NaN,NaN,NaN,2.0,5.0,1.0,0.040466,0.043091,0.015793,2.0,1,JGX202109300000007935151,4.0,1.0,4b345248ce,1.0,"78,110",1.0,2.0,0.0,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GRHLA20211629,0.0,1.0,1.0,1.0,0.0,0.0,0.0,112.0,2099-12-01,NaN,NaN,NaN,2.0,5.0,1.0,0.040466,0.043091,0.015793,2.0,1,JGX202109300000007935172,9.0,1.0,4b345248ce,1.0,"78,110",8.0,3.0,0.0,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075534 entries, 0 to 2075533
Data columns (total 39 columns):
 #   Column        Dtype         
---  ------        -----         
 0   prod_code     object        
 1   计价类型          float64       
 2   周期类型          float64       
 3   模式            float64       
 4   风险等级          float64       
 5   是否滚存          float64       
 6   是否允许变更分红方式    float64       
 7   产品品种          float64       
 8   持有天数          float64       
 9   数据日期          datetime64[ns]
 10  募集方式          float64       
 11  管理方式          float64       
 12  业务模式          float64       
 13  收益特点          float64       
 14  期限            float64       
 15  投资模式          float64       
 16  预期收益率         float16       
 17  最高收益率         float16       
 18  最低收益率         float16       
 19  展示等级          float16       
 20  prod_class    int64         
 21  流水号           object        
 22  业务代码          float64       
 23  渠道标识          float64       
 24

In [152]:
prod_df.nunique()

prod_code         25825
计价类型                  2
周期类型                  2
模式                    4
风险等级                  3
是否滚存                  2
是否允许变更分红方式            2
产品品种                  1
持有天数                320
数据日期                 32
募集方式                  2
管理方式                  2
业务模式                  2
收益特点                  2
期限                    6
投资模式                  3
预期收益率               249
最高收益率               215
最低收益率                34
展示等级                  5
prod_class            4
流水号             2052078
业务代码                 10
渠道标识                  5
core_cust_id      62074
净值                  157
申请金额              39122
资金状态                  8
交易状态                  5
总金额                1124
trade_date          273
模式2                   7
超额管理费               960
份额冻结比率                1
收益计息基数                3
产品面值                  2
发行价格                  2
折扣率                   7
费率                    2
dtype: int64

#### 产品品种、份额冻结比率，只有一个值，可以删除

In [153]:
del prod_df['产品品种']

In [154]:
del prod_df['份额冻结比率']

In [155]:
#A/B/C/D客户的重合度
a_num = A_df['core_cust_id'].value_counts().index.tolist()
b_num = B_df['core_cust_id'].value_counts().index.tolist()
c_num = C_df['core_cust_id'].value_counts().index.tolist()
d_num = D_df['core_cust_id'].value_counts().index.tolist()

In [156]:
len(set(a_num) & set(b_num)), len(set(a_num) & set(c_num)), len(set(a_num) & set(d_num)), 
len(set(b_num) & set(c_num)), len(set(d_num) & set(b_num)), len(set(c_num) & set(d_num))

(455, 6497, 754)

In [157]:
len(set(a_num) & set(b_num) & set(c_num) & set(d_num)), len(set(a_num) | set(b_num) | set(c_num) | set(d_num)) 

(230, 62074)

In [158]:
len(a_num), len(b_num), len(c_num), len(d_num)

(51888, 13678, 2852, 19012)

### D客户信息表

In [354]:
d_data = pd.read_csv(path2 + 'd.csv')
d_data = reduce_mem_usage(d_data)
d_data.head(2)

Memory usage of dataframe is 8.06 MB
Memory usage after optimization is: 3.02 MB
Decreased by 62.5%


,core_cust_id,d1,d2,d3
0,d4931873cb,1,4.0,40
1,af52580627,2,3.0,32


In [355]:
d_columns = ['core_cust_id', '性别', '客户等级', '年龄']

In [356]:
d_data.columns = d_columns
d_data.head(2)

,core_cust_id,性别,客户等级,年龄
0,d4931873cb,1,4.0,40
1,af52580627,2,3.0,32


In [357]:
d_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264055 entries, 0 to 264054
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   core_cust_id  264055 non-null  object 
 1   性别            264055 non-null  int8   
 2   客户等级          228810 non-null  float16
 3   年龄            264055 non-null  int8   
dtypes: float16(1), int8(2), object(1)
memory usage: 3.0+ MB


In [358]:
d_data.nunique()

core_cust_id    264055
性别                   2
客户等级                 4
年龄                  84
dtype: int64

In [359]:
d_data['客户等级'].value_counts()

4.0    164693
3.0     40932
2.0     21614
1.0      1571
Name: 客户等级, dtype: int64

In [362]:
d_data['年龄'].value_counts();

In [363]:
d_data.to_pickle('cust_info.pkl')

In [169]:
len(set(d_data['core_cust_id'].unique().tolist()) & using_cust)

264055

#### 客户信息表刚好契合data

### E客户风险表

In [364]:
e_data = pd.read_csv(path2 + 'e.csv')
e_data = reduce_mem_usage(e_data)
e_data.head(2)

Memory usage of dataframe is 7.24 MB
Memory usage after optimization is: 3.92 MB
Decreased by 45.8%


,core_cust_id,e1,e2
0,d4931873cb,3,20200608
1,af52580627,3,20200330


In [365]:
e_columns = ['core_cust_id', '客户风险等级', '评估日期']

In [366]:
e_data.columns = e_columns
e_data.head(2)

,core_cust_id,客户风险等级,评估日期
0,d4931873cb,3,20200608
1,af52580627,3,20200330


In [367]:
e_data['评估日期'] = e_data['评估日期'].astype('object')
e_data['评估日期'] = pd.to_datetime(e_data['评估日期'], format='%Y%m%d', errors='coerce')

In [368]:
e_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316116 entries, 0 to 316115
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   core_cust_id  316116 non-null  object        
 1   客户风险等级        316116 non-null  int8          
 2   评估日期          316116 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(1)
memory usage: 5.1+ MB


In [370]:
e_data.nunique()

core_cust_id    220823
客户风险等级               5
评估日期              2687
dtype: int64

In [371]:
e_data.to_pickle('cust_risk.pkl')

In [176]:
e_data['客户风险等级'].value_counts()

2    161543
3     92841
1     38793
4     15725
5      7214
Name: 客户风险等级, dtype: int64

In [177]:
#e_data['new_date'] = e_data.groupby(['core_cust_id'])['评估日期'].agg(max)

In [178]:
#e_data_id = e_data.groupby(['core_cust_id'])['new_date']
#e_data_id.count()

In [179]:
risk_data = pd.merge(e_data, buy_data, on='core_cust_id')

In [180]:
risk_data['客户风险等级'].value_counts()

2    32739
3    26333
1     6457
4     4399
5     2164
Name: 客户风险等级, dtype: int64

In [181]:
#客户风险等级，查看购买产品的用户风险等级是多少

In [182]:
len(set(e_data['core_cust_id'].unique().tolist()) & using_cust)

127495

#### 一部分不能用且有缺失

In [183]:
#buy_cust:16656
len(set(e_data['core_cust_id'].unique().tolist()) & set(buy_cust))

16585

In [184]:
#总共220823，可用127495， 缺失136560
e_using = list(set(e_data['core_cust_id'].unique().tolist()) & using_cust)

### F资产信息表

In [372]:
#去除千分位的逗号：df = df.applymap(lambda x: x.replace(',', ''))；或者用下面方法
f_data = pd.read_csv(path2 + 'f.csv', thousands=',')
f_data = reduce_mem_usage(f_data)
f_data.head(2)

Memory usage of dataframe is 182.25 MB
Memory usage after optimization is: 99.05 MB
Decreased by 45.7%


,core_cust_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22
0,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210830
1,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210730


In [373]:
f_columns = ['core_cust_id', '客户编号创建日期', '定期季日均', '大额存单季日均', 'A理财产品季日均', '代销资管季日均', '代销基金季日均', '定期时点余额',
             '大额存单时点余额', 'A理财产品时点余额', '代销资管时点余额', '代销基金时点余额', '定期月日均', '大额存单月日均', 'A理财产品月日均', 
             '代销资管月日均', '代销基金月日均', '定期年日均', '大额存单年日均', 'A理财产品年日均', '代销资管年日均', '代销基金年日均', '数据日期']


In [374]:
f_data.columns = f_columns
f_data.head(2)

,core_cust_id,客户编号创建日期,定期季日均,大额存单季日均,A理财产品季日均,代销资管季日均,代销基金季日均,定期时点余额,大额存单时点余额,A理财产品时点余额,代销资管时点余额,代销基金时点余额,定期月日均,大额存单月日均,A理财产品月日均,代销资管月日均,代销基金月日均,定期年日均,大额存单年日均,A理财产品年日均,代销资管年日均,代销基金年日均,数据日期
0,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210830
1,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210730


In [375]:
f_data['客户编号创建日期'] = f_data['客户编号创建日期'].astype('object')
f_data['客户编号创建日期'] = pd.to_datetime(f_data['客户编号创建日期'], errors='coerce')

In [376]:
f_data['数据日期'] = f_data['数据日期'].astype('object')
f_data['数据日期'] = pd.to_datetime(f_data['数据日期'], format='%Y%m%d', errors='coerce')

In [377]:
f_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038596 entries, 0 to 1038595
Data columns (total 23 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   core_cust_id  1038596 non-null  object        
 1   客户编号创建日期      1038596 non-null  datetime64[ns]
 2   定期季日均         97285 non-null    float32       
 3   大额存单季日均       102669 non-null   float32       
 4   A理财产品季日均      53939 non-null    float32       
 5   代销资管季日均       25147 non-null    float32       
 6   代销基金季日均       24650 non-null    float32       
 7   定期时点余额        97285 non-null    float32       
 8   大额存单时点余额      102669 non-null   float32       
 9   A理财产品时点余额     53939 non-null    float32       
 10  代销资管时点余额      25147 non-null    float32       
 11  代销基金时点余额      24650 non-null    float32       
 12  定期月日均         97285 non-null    float32       
 13  大额存单月日均       102669 non-null   float32       
 14  A理财产品月日均      53939 non-null    float32       
 15

In [191]:
f_data.nunique()

core_cust_id    264054
客户编号创建日期          1159
定期季日均            15240
大额存单季日均           4341
A理财产品季日均         21978
代销资管季日均           4793
代销基金季日均          15479
定期时点余额            6276
大额存单时点余额          1408
A理财产品时点余额        19440
代销资管时点余额          4034
代销基金时点余额         14351
定期月日均             9012
大额存单月日均           2365
A理财产品月日均         20675
代销资管月日均           4389
代销基金月日均          14960
定期年日均            35716
大额存单年日均          10099
A理财产品年日均         26214
代销资管年日均           6455
代销基金年日均          16377
数据日期                 4
dtype: int64

In [192]:
f_data[~f_data['A理财产品年日均'].isnull()].nunique()

core_cust_id    13583
客户编号创建日期          917
定期季日均            3113
大额存单季日均          1300
A理财产品季日均        21978
代销资管季日均          1726
代销基金季日均          4688
定期时点余额           1430
大额存单时点余额          684
A理财产品时点余额       19440
代销资管时点余额         1429
代销基金时点余额         4346
定期月日均            2020
大额存单月日均           954
A理财产品月日均        20675
代销资管月日均          1571
代销基金月日均          4519
定期年日均            7061
大额存单年日均          2574
A理财产品年日均        26214
代销资管年日均          2489
代销基金年日均          5235
数据日期                4
dtype: int64

#### 和data差一个人的记录

In [193]:
len(set(f_data['core_cust_id'].unique().tolist()) & using_cust)

264054

### R：app点击行为表

In [194]:
r_data = pd.read_csv(path2 + 'r.csv')
r_data.head()

,r1,core_cust_id,r3,prod_code,r5
0,20210825_11277467902,a15a1d681a,2,91318017,2021-08-25 14:18:10
1,20210824_11229966502,a15a1d681a,1,GRHLA20211386,2021-08-24 14:55:49
2,20210824_11234138402,a15a1d681a,1,GRHLA20211386,2021-08-24 16:17:15
3,20210824_11229929502,a15a1d681a,1,GRHLA20211386,2021-08-24 14:55:10
4,20210824_11230210002,a15a1d681a,1,GRHLA20211386,2021-08-24 15:00:02


In [195]:
r_columns = ['id编码', 'core_cust_id', '点击事件类型', 'prod_code', '点击时间']

In [196]:
r_data.columns = r_columns
r_data.head(2)

,id编码,core_cust_id,点击事件类型,prod_code,点击时间
0,20210825_11277467902,a15a1d681a,2,91318017,2021-08-25 14:18:10
1,20210824_11229966502,a15a1d681a,1,GRHLA20211386,2021-08-24 14:55:49


In [197]:
r_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777075 entries, 0 to 777074
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id编码          777075 non-null  object
 1   core_cust_id  777075 non-null  object
 2   点击事件类型        777075 non-null  int64 
 3   prod_code     777068 non-null  object
 4   点击时间          777075 non-null  object
dtypes: int64(1), object(4)
memory usage: 29.6+ MB


In [198]:
r_data.nunique()

id编码            777075
core_cust_id     55864
点击事件类型               2
prod_code         1193
点击时间            739817
dtype: int64

In [199]:
#buy_cust:16656
len(set(r_data['core_cust_id'].unique().tolist()) & set(buy_cust))

13605

In [200]:
len(set(r_data['core_cust_id'].unique().tolist()) & using_cust)

51633

#### 部分可用，缺失多

- 只有51633个用户记录，缺少212422个用户记录

### S账户交易流水表

In [201]:
s_data = pd.read_csv(path2 + 's.csv', thousands=',')
s_data = reduce_mem_usage(s_data)
s_data.head(2)

Memory usage of dataframe is 347.56 MB
Memory usage after optimization is: 279.29 MB
Decreased by 19.6%


,s1,s2,s3,s4,s5,s6,s7
0,01004320210415202104151FT21105818179741,4,NaN,75617.812500,2021-04-15,9809df0ffe,2021-04-15
1,01004320210630202106301FT21181810270081,4,NaN,75635.101562,2021-06-30,9809df0ffe,2021-06-30


In [202]:
val_str = str()
s_data['s3'].fillna(val_str, inplace=True)
s_data['s6'].fillna(val_str, inplace=True)
s_data['core_cust_id'] = s_data['s3'] + s_data['s6']
s_data['core_cust_id'] = s_data['core_cust_id'].apply(lambda x:x[:9])
del s_data['s3'], s_data['s6']

In [203]:
s_data.head(2)

,s1,s2,s4,s5,s7,core_cust_id
0,01004320210415202104151FT21105818179741,4,75617.812500,2021-04-15,2021-04-15,9809df0ff
1,01004320210630202106301FT21181810270081,4,75635.101562,2021-06-30,2021-06-30,9809df0ff


In [204]:
s_columns = ['事件编号', '交易类型', '借方金额', 's交易日期', 's处理日期', 'core_cust_id']

In [205]:
s_data.columns = s_columns

In [206]:
s_data.head(3)

,事件编号,交易类型,借方金额,s交易日期,s处理日期,core_cust_id
0,01004320210415202104151FT21105818179741,4,75617.812500,2021-04-15,2021-04-15,9809df0ff
1,01004320210630202106301FT21181810270081,4,75635.101562,2021-06-30,2021-06-30,9809df0ff
2,01004320210205202102051FT21036609489561,4,75955.625000,2021-02-05,2021-02-05,9809df0ff


In [207]:
s_data.nunique()

事件编号            6507904
交易类型                 12
借方金额             328812
s交易日期               273
s处理日期               273
core_cust_id      96086
dtype: int64

In [208]:
s_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6507904 entries, 0 to 6507903
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   事件编号          object 
 1   交易类型          int8   
 2   借方金额          float32
 3   s交易日期         object 
 4   s处理日期         object 
 5   core_cust_id  object 
dtypes: float32(1), int8(1), object(4)
memory usage: 229.6+ MB


In [209]:
#buy_cust:16656
len(set(s_data['core_cust_id'].unique().tolist()) & set(buy_cust))

2346

In [210]:
len(set(s_data['core_cust_id'].unique().tolist()) & using_cust)

12611

- 训练测试集cust：264055， 账户交易流水表中cust：96086；交集12611

### 总结

- A、B、C、D四个表单独合并，流水表缺失用户20万(59372)，总用户26万(264055),购买用户却全部有记录（预处理删除一些不存在流水表中的用户数据）
- D客户信息表可以直接合并
- F资产信息表可以直接合并，缺失一条记录，对比看在train和test哪个里面
- E客户风险表，缺失13万个用户信息;.......16656/16585,客户风险表中包含大部分购买人信息。。。。。
- R点击行为，缺少21万个用户的记录;。。。。。1.6万购买人中1.3万人有点击行为记录
- S账户交易流水， 缺少25万个用户的交易流水

### 合并所有表（不包括S）

#### 合并train/test/D客户信息表/客户风险表/F资产信息表/APP点击表

合并train和test

In [218]:
df_tt = pd.concat([train, test])

In [219]:
df_tt.head(2)

,id,core_cust_id,prod_code,y,type,trade_date,prod_class
0,4e3c3d57b83e425f8087b1d6d32a50f7,6e2105d9fe,90318011,0.0,train,2021-08-01,4
1,aa83c5fc05414c4d9727f0b32882f80e,6e2105d9fe,GRHLA20211530,0.0,train,2021-09-01,1


In [220]:
df_tt.shape

(2953495, 7)

In [353]:
df_tt.to_pickle('data.pkl')

In [226]:
df = pd.merge(df_tt, record, on=['core_cust_id', 'prod_code', 'prod_class', 'trade_date'], how='left')

In [228]:
df.shape

(2955246, 14)

In [227]:
df.isnull().sum()

id                    0
core_cust_id          0
prod_code             0
y                567362
type                  0
trade_date            0
prod_class            0
流水号             2941611
业务代码            2941611
渠道标识            2941611
净值              2941611
申请金额            2941611
资金状态            2941611
交易状态            2941611
dtype: int64

合并train/test和D客户信息表

In [214]:
df_ttd = pd.merge(df_tt, d_data, on='core_cust_id', how='left')

In [215]:
df_ttd.shape

(2953495, 10)

合并train/test和E客户风险表

In [216]:
e_data.head()

,core_cust_id,客户风险等级,评估日期
0,d4931873cb,3,2020-06-08
1,af52580627,3,2020-03-30
2,cfeaff22c6,1,2020-04-26
3,a15a0e01ea,2,2019-10-10
4,7c72b6e583,2,2021-08-09


In [217]:
e_data[e_data['core_cust_id'] == '7c72b6e583']

,core_cust_id,客户风险等级,评估日期
4,7c72b6e583,2,2021-08-09


In [218]:
e_data_7 = e_data[e_data['评估日期']<'2021-07-01']
e_data_7.head(2)

,core_cust_id,客户风险等级,评估日期
0,d4931873cb,3,2020-06-08
1,af52580627,3,2020-03-30


In [219]:
e_data_7.sort_values(by='评估日期', inplace=True)

In [220]:
e_data_8 = e_data[e_data['评估日期'] < '2021-08-01']
e_data_8 = e_data_8[e_data_8['评估日期'] >= '2021-07-01']
e_data_8.sort_values(by='评估日期', inplace=True)

e_data_9 = e_data[e_data['评估日期']<'2021-09-01']
e_data_9 = e_data_9[e_data_9['评估日期'] >= '2021-08-01']
e_data_9.sort_values(by='评估日期', inplace=True)

e_data_10 = e_data[e_data['评估日期'] >= '2021-09-01']
e_data_10.sort_values(by='评估日期', inplace=True)

In [221]:
e_data_7.drop_duplicates('core_cust_id', 'last', inplace=True)
e_data_8.drop_duplicates('core_cust_id', 'last', inplace=True)
e_data_9.drop_duplicates('core_cust_id', 'last', inplace=True)
e_data_10.drop_duplicates('core_cust_id', 'last', inplace=True)

In [222]:
e_data_7['trade_date'] = '2021-07-01'
e_data_8['trade_date'] = '2021-08-01'
e_data_9['trade_date'] = '2021-09-01'
e_data_10['trade_date'] = '2021-10-01'

In [223]:
risk_df = pd.concat([e_data_7, e_data_8, e_data_9, e_data_10])

In [224]:
risk_df.reset_index(inplace=True)

In [225]:
risk_df.head(3)

,index,core_cust_id,客户风险等级,评估日期,trade_date
0,4691,e33e234fc,3,2013-03-04,2021-07-01
1,300167,4e3b2d675,3,2013-03-04,2021-07-01
2,138060,4e3b2e03b,4,2013-03-05,2021-07-01


In [226]:
risk_df['trade_date'] = pd.to_datetime(risk_df['trade_date'], errors='coerce')

In [227]:
risk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229965 entries, 0 to 229964
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   index         229965 non-null  int64         
 1   core_cust_id  229965 non-null  object        
 2   客户风险等级        229965 non-null  int8          
 3   评估日期          229965 non-null  datetime64[ns]
 4   trade_date    229965 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), int8(1), object(1)
memory usage: 7.2+ MB


In [228]:
df_ttde = pd.merge(df_ttd, risk_df, on=['core_cust_id', 'trade_date'], how='left')

In [229]:
df_ttde.shape

(2953495, 13)

In [230]:
f_data.head(2)

,core_cust_id,客户编号创建日期,定期季日均,大额存单季日均,A理财产品季日均,代销资管季日均,代销基金季日均,定期时点余额,大额存单时点余额,A理财产品时点余额,代销资管时点余额,代销基金时点余额,定期月日均,大额存单月日均,A理财产品月日均,代销资管月日均,代销基金月日均,定期年日均,大额存单年日均,A理财产品年日均,代销资管年日均,代销基金年日均,数据日期
0,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-30
1,48e055617a,2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-30


In [231]:
f_data_7 = f_data[f_data['数据日期'] < '2021-07-01']
f_data_7.sort_values(by='数据日期', inplace=True)

f_data_8 = f_data[f_data['数据日期'] < '2021-08-01']
f_data_8 = f_data_8[f_data_8['数据日期'] >= '2021-07-01']
f_data_8.sort_values(by='数据日期', inplace=True)

f_data_9 = f_data[f_data['数据日期']<'2021-09-01']
f_data_9 = f_data_9[f_data_9['数据日期'] >= '2021-08-01']
f_data_9.sort_values(by='数据日期', inplace=True)

f_data_10 = f_data[f_data['数据日期'] >= '2021-09-01']
f_data_10.sort_values(by='数据日期', inplace=True)

In [232]:
f_data_7.drop_duplicates('core_cust_id', 'last', inplace=True)
f_data_8.drop_duplicates('core_cust_id', 'last', inplace=True)
f_data_9.drop_duplicates('core_cust_id', 'last', inplace=True)
f_data_10.drop_duplicates('core_cust_id', 'last', inplace=True)

In [233]:
f_data_7['trade_date'] = '2021-07-01'
f_data_8['trade_date'] = '2021-08-01'
f_data_9['trade_date'] = '2021-09-01'
f_data_10['trade_date'] = '2021-10-01'

In [234]:
asset_df = pd.concat([f_data_7, f_data_8, f_data_9, f_data_10])
asset_df.reset_index(inplace=True)

In [235]:
asset_df['trade_date'] = pd.to_datetime(asset_df['trade_date'], errors='coerce')

In [236]:
asset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038596 entries, 0 to 1038595
Data columns (total 25 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   index         1038596 non-null  int64         
 1   core_cust_id  1038596 non-null  object        
 2   客户编号创建日期      1038596 non-null  datetime64[ns]
 3   定期季日均         97285 non-null    float32       
 4   大额存单季日均       102669 non-null   float32       
 5   A理财产品季日均      53939 non-null    float32       
 6   代销资管季日均       25147 non-null    float32       
 7   代销基金季日均       24650 non-null    float32       
 8   定期时点余额        97285 non-null    float32       
 9   大额存单时点余额      102669 non-null   float32       
 10  A理财产品时点余额     53939 non-null    float32       
 11  代销资管时点余额      25147 non-null    float32       
 12  代销基金时点余额      24650 non-null    float32       
 13  定期月日均         97285 non-null    float32       
 14  大额存单月日均       102669 non-null   float32       
 15

In [237]:
df_tdef = pd.merge(df_ttde, asset_df, on=['core_cust_id', 'trade_date'], how='left')

In [238]:
df_tdef.shape

(2953495, 36)

合并R：APP点击行为表

In [239]:
r_data.head(2)

,id编码,core_cust_id,点击事件类型,prod_code,点击时间
0,20210825_11277467902,a15a1d681a,2,91318017,2021-08-25 14:18:10
1,20210824_11229966502,a15a1d681a,1,GRHLA20211386,2021-08-24 14:55:49


In [241]:
r_pivot_stat = pd.pivot_table(r_data, index='core_cust_id', columns=['点击事件类型'], values=['id编码'], aggfunc='count').fillna(0)
r_pivot_stat = r_pivot_stat.reset_index()
r_pivot_stat.columns = ['core_cust_id', 'uid_cnt_in_r3_equal_1', 'uid_cnt_in_r3_equal_2']
df_tdef = df_tdef.merge(r_pivot_stat, on='core_cust_id', how='left')

r_stat = r_data.groupby(['core_cust_id']).agg({
                                         'prod_code':['nunique'],
                                         '点击时间':['count']}
                                         ).reset_index()
r_stat.columns = ['core_cust_id','prod_code_nunique_grp_uid_in_app_action','r5_nunique_grp_uid_in_app_action']
df_defr = df_tdef.merge(r_stat, on='core_cust_id', how='left')

In [242]:
#无ABCD交易流水记录
df_no = df_defr.merge(prod_data, on='prod_code', how='left') 

In [243]:
df_no.shape

(2953495, 66)

In [244]:
df_no[:50000].to_pickle('df_no.pkl')

In [246]:
df_no.to_pickle('df_no.pkl')

In [247]:
df_no.to_csv('df_no.csv', index=False)